In [1]:
import pandas as pd
import numpy as np

In [ ]:
# description

In [5]:
#load data

# load locs:
locs = pd.read_csv('20200902_locs.csv')
# load wahlberg2009
wahlberg2009 = pd.read_csv('20200902_QTLWahlberg2009.csv')
# load jacobsson2005
jacobsson2005 = pd.read_csv('20200902_QTLjacobsson2005.csv')

In [15]:
# dummy vars
cm = 442
chrom=

,Unnamed: 0,Chromosome,cM,GalGal5Mb,GalGal6Mb,GalGal3Mb,Marker
0,1,1,0.0,0.3,0.3,0.3,RBL438
1,2,1,36.0,0.8,1.0,0.7,rs14080307
2,3,1,41.7,2.6,2.8,2.4,rs13824562
3,4,1,47.5,6.0,5.9,6.1,rs13621730
4,5,1,57.1,12.8,11.9,13.8,rs14795085
...,...,...,...,...,...,...,...
400,401,28,0.0,0.7,0.7,0.7,rs15245588
401,402,28,33.7,2.6,3.5,2.1,rs14306112
402,403,28,37.7,3.0,3.9,2.4,rs16211629
403,404,28,41.5,NaN,NaN,2.5,MCW0227


In [68]:
def _get_flanking_marker(cM, chrom, locs, side='left'):
    if side=='left':
        # get all flanking markers:
        all_left = locs.loc[(locs["cM"] < cm) & (locs["Chromosome"] == chrom)]
        # get the closest left flanking marker
        closest_flank = all_left.loc[all_left['cM']==np.max(all_left['cM'])].iloc[0] 
        
    if side=='right':
        # get all flanking markers:
        all_left = locs.loc[(locs["cM"] > cm) & (locs["Chromosome"] == chrom)]
        # get the closest left flanking marker
        closest_flank = all_left.loc[all_left['cM']==np.min(all_left['cM'])].iloc[0]
        
    return closest_flank

In [37]:
locs.loc[(locs["cM"] < cm) & (locs["Chromosome"] == chrom)]

,Unnamed: 0,Chromosome,cM,GalGal5Mb,GalGal6Mb,GalGal3Mb,Marker
0,1,1,0.0,0.3,0.3,0.3,RBL438
1,2,1,36.0,0.8,1.0,0.7,rs14080307
2,3,1,41.7,2.6,2.8,2.4,rs13824562
3,4,1,47.5,6.0,5.9,6.1,rs13621730
4,5,1,57.1,12.8,11.9,13.8,rs14795085
...,...,...,...,...,...,...,...
58,59,1,407.7,153.0,148.8,157.1,rs14905876
59,60,1,410.6,154.3,150.1,158.4,LEI0079
60,61,1,426.3,164.8,160.7,169.1,LEI0162
61,62,1,434.6,167.4,163.2,171.7,ADL0245


In [69]:
a = _get_flanking_marker(cM=cm,chrom=1, locs=locs, side='left')
b = _get_flanking_marker(cM=cm,chrom=1, locs=locs, side='right')

In [66]:
a['Marker']

'rs13621730'

In [51]:
#wahlberg2009

In [48]:
b

,Unnamed: 0,Chromosome,cM,GalGal5Mb,GalGal6Mb,GalGal3Mb,Marker
63,64,1,448.0,170.9,166.6,175.2,rs15502284


In [71]:
# get gg6 positions for left and right markers of wahlberg_qtls
leftmark = []
leftpos = []
rightmark = []
rightpos = []

for i, k in wahlberg2009.iterrows():
    cm = k['cM']
    chrom = k['Chromosome']
    left_flank = _get_flanking_marker(cM=cm,chrom=chrom, locs=locs, side='left')
    right_flank = _get_flanking_marker(cM=cm,chrom=chrom, locs=locs, side='right')
    leftmark.append(left_flank['Marker'])
    leftpos.append(left_flank['GalGal6Mb'])
    
    rightmark.append(right_flank['Marker'])
    rightpos.append(right_flank['GalGal6Mb'])

    #print('left-flank marker is {mrk} at {mb} Mb'.format(mrk=left_flank['Marker'],mb=left_flank['GalGal6Mb']))
    #print('right-flank marker is {} at {} Mb')

In [73]:
wahlberg2009['leftmark']=leftmark
wahlberg2009['rightmark']=rightmark
wahlberg2009['leftpos']=leftpos
wahlberg2009['rightpos']=rightpos

In [96]:
locs

,Unnamed: 0,Chromosome,cM,GalGal5Mb,GalGal6Mb,GalGal3Mb,Marker
0,1,1,0.0,0.3,0.3,0.3,RBL438
1,2,1,36.0,0.8,1.0,0.7,rs14080307
2,3,1,41.7,2.6,2.8,2.4,rs13824562
3,4,1,47.5,6.0,5.9,6.1,rs13621730
4,5,1,57.1,12.8,11.9,13.8,rs14795085
...,...,...,...,...,...,...,...
400,401,28,0.0,0.7,0.7,0.7,rs15245588
401,402,28,33.7,2.6,3.5,2.1,rs14306112
402,403,28,37.7,3.0,3.9,2.4,rs16211629
403,404,28,41.5,NaN,NaN,2.5,MCW0227


In [98]:
three_three = []
for i,l in locs.iterrows():
    a,b = l['Marker'][:3],l['Marker'][-3:]
    for j, k in jacobsson2005.iterrows():
        jacL3 =  k['MrkL'][:3]
        jacLm3 = k['MrkL'][-3:]
        if (jacL3 == a) & (jacLm3==b):
            print(l.Marker)
    
    jacR3 =  k['MrkR'][:3]
    jacRm3 = k['MrkR'][-3:]
        if (jacR3 == a) & (jacRm3==b):
            print(l.Marker)

LEI0079
MCW0063
LEI0147
MCW0222
MCW0224
MCW0224
ADL0317
ADL0317
ADL0317
ADL0317
ADL0317
ADL0317
LEI0125
LEI0125
LEI0125
LEI0125
LEI0125
MCW0038
MCW0120
MCW0120
MCW0120
MCW0120
MCW0120
MCW0120
ROS00325
ROS00325
ROS00325
MCW0119
ADL0125
ADL0125
MCW0227


In [107]:
lmarks = []
lposs = []
rposs = []
rmarks = []
for j, k in jacobsson2005.iterrows():
    lmark = 'NA'
    rmark = 'NA'
    rpos = 'NA'
    lpos = "NA"
    jacL3 =  k['MrkL'][:3]
    jacLm3 = k['MrkL'][-3:]
    for i,l in locs.iterrows():
        a,b = l['Marker'][:3],l['Marker'][-3:]
        if (jacL3 == a) & (jacLm3==b):
            lmark = l['Marker']
            lpos = l['GalGal6Mb']
            
    jacR3 =  k['MrkR'][:3]
    jacRm3 = k['MrkR'][-3:]
    for i,l in locs.iterrows():
        a,b = l['Marker'][:3],l['Marker'][-3:]
        if (jacR3 == a) & (jacRm3==b):
            rmark = l['Marker']
            rpos = l['GalGal6Mb']
    lmarks.append(lmark)
    rmarks.append(rmark)
    rposs.append(rpos)
    lposs.append(lpos)

In [108]:
jacobsson2005['lmark'] = lmarks
jacobsson2005['rmark'] = rmarks
jacobsson2005['lpos'] = lposs
jacobsson2005['rpos'] = rposs


In [112]:
jacobsson2005.to_csv('20200903_QTLjacobsson2005_with_positions.csv')

In [83]:
wahlberg2009.to_csv('20200902_QTLWahlberg2009_with_positions.csv')